In [43]:
import pandas as pd
covid_data = pd.read_csv('data/covid_data.csv')
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

## 3. Знакомимся с новыми данными: коронавирус


Мы будем изучать методы визуализации, используя два набора данных:

1. [covid_data](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e8e401d5f46b6c4e81f7330cc24d4fe2/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/covid_data.zip) — данные о показателях заболеваемости за каждый день в различных регионах 187 стран. [(источник данных)](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)

Мы будем работать со следующими столбцами:

* *date* — дата наблюдения;
* *province*/*state* — наименование провинции/штата;
* *country* — наименование страны;
* *confirmed* — общее число зафиксированных случаев на указанный день;
* *deaths* — общее число зафиксированных смертей на указанный день;
* *recovered* — общее число выздоровлений на указанный день

2. [country_vaccinations](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4f12d963943a898af014981a4a822088/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/country_vaccinations.zip) — данные о процессе вакцинирования людей в 221 стране мира. [(источник данных)](https://www.kaggle.com/gpreda/covid-world-vaccination-progress)

> Обратите внимание, что в приведённом источнике данные о вакцинации постоянно обновляются. В данном модуле будет использоваться файл, который мы скачали заранее.

Данная таблица содержит следующие столбцы:

* *country* — наименование страны;
* *date* — дата наблюдения;
* *total_vaccinations* — общее число введённых вакцин в стране на указанный день;
* *people_vaccinated* — общее число привитых первым компонентом в стране на указанный день;
* *people_vaccinated_per_hundred* — процент привитых первым компонентом в стране на указанный день (рассчитывается как *people_vaccinated / population* * 100);
* *people_fully_vaccinated* — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
* *people_fully_vaccinated_per_hundred* — процент привитых вторым компонентом в стране на указанный день (рассчитывается как *people_fully_vaccinated* / *population* * 100);
* *daily_vaccination* — ежедневная вакцинация (число вакцинированных в указанный день);
* *vaccines* — комбинации вакцин, используемые в стране.

#### НЕМНОГО ОБ ОСОБЕННОСТЯХ ДАННЫХ
В таблице *covid_data* данные о заболеваемости представлены для каждого региона (штата) страны за каждый день.    
Например, для США (United States) записи за 29 мая 2021 года имеют вид:   

![таблица](https://lms-cdn.skillfactory.ru/assets/courseware/v1/230393ba3d8d6c83c12538a8c4f1d0c6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_4.png)

В таблице *vaccinations_data* параметры вакцинирования отображены для всех стран на каждый день, без учёта регионов.    
Вот часть таблицы для тех же Соединённых Штатов за 29 мая 2021 года. Как видите, она всего одна:

![таблица](https://lms-cdn.skillfactory.ru/assets/courseware/v1/83b7cc0f85531b6c2f07146579fab466/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_5.png)

В данных много пропусков, а также неверно введённых значений. Например, для США (United States) и Великобритании (United Kingdom) в большинстве записей таблицы covid_data число выздоровевших людей равно 0.    
В данном модуле мы закроем на это глаза, но вообще эти страны подлежали бы исключению из данных или нам понадобился бы дополнительный источник данных, из которого мы смогли бы получить недостающую информацию.

> братите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти.    
Такие признаки называют накопительными.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1. В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.
2. В таблицах не совпадает число стран, а иногда и их названия. 
При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны).    
Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.
3. Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

#### ПРЕДОБРАБОТКА ДАННЫХ
Опираясь на замечания выше, выполним небольшую предобработку.

В таблице *covid_data*:

1. Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [44]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

2. Преобразуем даты в формат datetime с помощью функции **pd.to_datetime()**:

In [45]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

3. Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

In [46]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

4. Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам.   
После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода *diff*():

In [47]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

Итоговый вид таблицы *covid_data*:

![covid_data](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c372c51b2676436625da5f1402faa1f1/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md13_3_6.png)

В таблице *vaccinations_data* достаточно будет преобразовать столбцы в формат *datetime*:

In [48]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

## Practice.3

### Задание 3.1 - 3.2
За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).
За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.

In [49]:
print(f"Данные по заболеваемости представленны за период с {covid_data['date'].min()} по {covid_data['date'].max()}")
print(f"Данные по вакцинации представленны за период с {vaccinations_data['date'].min()} по {vaccinations_data['date'].max()}")


Данные по заболеваемости представленны за период с 2020-01-22 00:00:00 по 2021-05-29 00:00:00
Данные по вакцинации представленны за период с 2020-12-02 00:00:00 по 2021-09-06 00:00:00


### Задание 3.3
С помощью метода *merge*() объедините таблицы *covid_data* и *vaccinations_data* по столбцам *date* и *country*.   
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1.    
То есть в результирующую таблицу должны попасть все записи из таблицы *covid_data* и из её пересечения с *vaccinations_data*, но не более.    
Результат объединения занесите в переменную ***covid_df***.

In [51]:
covid_df = covid_data.merge(
    vaccinations_data,
    on=['date', 'country'],
    how='left'    
)

print(f"Число строк {covid_df.shape[0]}")
print(f"Число столбцов {covid_df.shape[1]}")

Число строк 86785
Число столбцов 16


В получившейся в задании 3.3 таблице *covid_df* создайте признаки *death_rate* — общий процент смертей среди зафиксированных случаев (летальность) и *recover_rate* — общий процент случаев выздоровления.   
Данные характеристики рассчитайте как отношение числа смертей (*deaths*) и числа выздоровлений (*recovered*) к числу зафиксированных случаев (*confirmed*) и умножьте результаты на 100%.

In [63]:
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

### Задание 3.4

Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [61]:
print(f"Максимальная летальность в США равна : {round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)}")

Максимальная летальность в США равна : 7.19


### Задание 3.5
Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.


In [65]:
print(f"Средний процент выздоровивших в России равен: {round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)}")

Средний процент выздоровивших в России равен: 67.06
